In [4]:
import requests
import csv

In [5]:
# parse arguments
input_file = "../examples/run_input.csv"
output_file = "../examples/run_output_v2.csv"

# read smiles list
smiles_list = []
with open(input_file, "r") as f:
    reader = csv.reader(f)
    next(reader)
    for r in reader:
        smiles_list += [r[0]]

In [6]:
smiles_list

['COC(=O)[C@@H](N)CS',
 'CCCCNc1ccc(cc1)C(=O)OCCOCCOCCOCCOCCOCCOCCOCCOCCOC',
 'CCOCCOC(=O)\\C=C\\c1ccc(OC)cc1']

In [82]:
import requests

baseUrl = 'https://admetlab3.scbdd.com'

if __name__ == '__main__':
    api = '/api/single/admet'
    url = baseUrl + api
    param = {'SMILES': "CCC"}

    response = requests.post(url, json=param)
    print(response)
    if response.status_code == 200:
        data = response.json()['data']
        print(data)


<Response [500]>


In [83]:
response.reason

'Internal Server Error'